# Forecasting with SVR using new stats features

In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [23]:
basedir = 'C:/Users/jesus/Documents/School/College/Computer Science/Class/CS 4961 - Boeing MPS/repo/tlm-data/train/generated/engineered/'
file = 'Battery1.csv'
path = f'{basedir}{file}'

odasdf = pd.read_csv(path)
odasdf.tail()

,Time,Value,Avg3,Stdev3,Minimum3,Maximum3,Fluctuation3,Avg5,Stdev5,Minimum5,...,Avg7,Stdev7,Minimum7,Maximum7,Fluctuation7,Hour,Minute,Second,Microsecond,Unnamed: 21
6888,2020-02-05T08:00:07.204715,13.0,12.5,0.5196152422706635,11.9,12.8,0.9000000000000004,12.38,0.4266145801540311,11.9,...,12.142857142857144,0.5533448076836687,11.3,12.8,1.5,8,0,7,204715,NaN
6889,2020-02-05T08:00:07.305414,11.8,12.566666666666666,0.5859465277082315,11.9,13.0,1.0999999999999996,12.5,0.5099019513592786,11.9,...,12.314285714285715,0.612178000880385,11.3,13.0,1.6999999999999993,8,0,7,305414,NaN
6890,2020-02-05T08:00:07.406250,11.3,12.533333333333333,0.6429100507328634,11.8,13.0,1.1999999999999993,12.46,0.5639148871948673,11.8,...,12.385714285714286,0.4913538149119954,11.8,13.0,1.1999999999999993,8,0,7,406250,NaN
6891,2020-02-05T08:00:07.506779,11.3,12.033333333333333,0.87368949480541,11.3,13.0,1.6999999999999993,12.16,0.7162401831787991,11.3,...,12.22857142857143,0.6395683067879606,11.3,13.0,1.6999999999999993,8,0,7,506779,NaN
6892,2020-02-05T08:00:07.607320,12.5,11.466666666666667,0.28867513459481287,11.3,11.8,0.5,12.040000000000001,0.8142481194328911,11.3,...,12.12857142857143,0.7296444013222826,11.3,13.0,1.6999999999999993,8,0,7,607320,NaN


In [28]:
feat = []
for i in 3, 5, 7:
    feat.append(f'Avg{i}')
    feat.append(f'Stdev{i}')
    feat.append(f'Minimum{i}')
    feat.append(f'Maximum{i}')
    feat.append(f'Fluctuation{i}')
feat.append('Hour')
feat.append('Minute')
feat.append('Second')
feat.append('Microsecond')
print(feat)

label = 'Value'

['Avg3', 'Stdev3', 'Minimum3', 'Maximum3', 'Fluctuation3', 'Avg5', 'Stdev5', 'Minimum5', 'Maximum5', 'Fluctuation5', 'Avg7', 'Stdev7', 'Minimum7', 'Maximum7', 'Fluctuation7', 'Hour', 'Minute', 'Second', 'Microsecond']


In [31]:
split = .7
rows = len(odasdf)

train_size = int( rows * split )
print(train_size)

4825


In [37]:
X_train = odasdf[feat][7:train_size]
X_train.head()

,Avg3,Stdev3,Minimum3,Maximum3,Fluctuation3,Avg5,Stdev5,Minimum5,Maximum5,Fluctuation5,Avg7,Stdev7,Minimum7,Maximum7,Fluctuation7,Hour,Minute,Second,Microsecond
7,11.233333333333333,0.23094010767585052,11.1,11.5,0.40000000000000036,11.34,0.26076809620810587,11.1,11.7,0.5999999999999996,11.485714285714286,0.4980915960897535,11.1,12.5,1.4000000000000004,7,48,34,226037
8,11.533333333333333,0.4509249752822896,11.1,12.0,0.9000000000000004,11.48,0.3898717737923586,11.1,12.0,0.9000000000000004,11.414285714285715,0.33876526498728404,11.1,12.0,0.9000000000000004,7,48,34,326727
9,11.666666666666666,0.28867513459481287,11.5,12.0,0.5,11.44,0.3714835124201344,11.1,12.0,0.9000000000000004,11.457142857142857,0.325868802112869,11.1,12.0,0.9000000000000004,7,48,34,427272
10,11.633333333333333,0.3214550253664317,11.4,12.0,0.5999999999999996,11.5,0.3240370349203931,11.1,12.0,0.9000000000000004,11.471428571428572,0.31997023671109226,11.1,12.0,0.9000000000000004,7,48,34,527970
11,11.633333333333333,0.3214550253664317,11.4,12.0,0.5999999999999996,11.68,0.29495762407505244,11.4,12.0,0.5999999999999996,11.514285714285714,0.3716116764786034,11.1,12.0,0.9000000000000004,7,48,34,628777


In [38]:
y_train = odasdf[label][7:train_size]
y_train.head()

,Value
7,12.0
8,11.5
9,11.4
10,12.0
11,11.1


In [40]:
X_test = odasdf[feat][train_size:]
X_test.tail()

,Avg3,Stdev3,Minimum3,Maximum3,Fluctuation3,Avg5,Stdev5,Minimum5,Maximum5,Fluctuation5,Avg7,Stdev7,Minimum7,Maximum7,Fluctuation7,Hour,Minute,Second,Microsecond
6888,12.5,0.5196152422706635,11.9,12.8,0.9000000000000004,12.38,0.4266145801540311,11.9,12.8,0.9000000000000004,12.142857142857144,0.5533448076836687,11.3,12.8,1.5,8,0,7,204715
6889,12.566666666666666,0.5859465277082315,11.9,13.0,1.0999999999999996,12.5,0.5099019513592786,11.9,13.0,1.0999999999999996,12.314285714285715,0.612178000880385,11.3,13.0,1.6999999999999993,8,0,7,305414
6890,12.533333333333333,0.6429100507328634,11.8,13.0,1.1999999999999993,12.46,0.5639148871948673,11.8,13.0,1.1999999999999993,12.385714285714286,0.4913538149119954,11.8,13.0,1.1999999999999993,8,0,7,406250
6891,12.033333333333333,0.87368949480541,11.3,13.0,1.6999999999999993,12.16,0.7162401831787991,11.3,13.0,1.6999999999999993,12.22857142857143,0.6395683067879606,11.3,13.0,1.6999999999999993,8,0,7,506779
6892,11.466666666666667,0.28867513459481287,11.3,11.8,0.5,12.040000000000001,0.8142481194328911,11.3,13.0,1.6999999999999993,12.12857142857143,0.7296444013222826,11.3,13.0,1.6999999999999993,8,0,7,607320


In [41]:
y_test = odasdf[label][train_size:]
y_test.tail()

,Value
6888,13.0
6889,11.8
6890,11.3
6891,11.3
6892,12.5


In [45]:
svr = SVR(C=1.0, epsilon=0.2)
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print('RMSE is', rmse)

C:\Programs\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Programs\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


RMSE is 0.5836651770197268
